In [52]:
import torch
from tqdm.auto import tqdm

from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud, plot_3D

from PIL import Image
from tqdm.auto import tqdm
import plotly.graph_objects as go

from point_e.util.point_cloud import PointCloud


In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
%cd ..
!pip install -e .

c:\Users\X1G6\OneDrive - Hanoi University of Science and Technology\FPT_AIC\code
Obtaining file:///C:/Users/X1G6/OneDrive%20-%20Hanoi%20University%20of%20Science%20and%20Technology/FPT_AIC/code


ERROR: file:///C:/Users/X1G6/OneDrive%20-%20Hanoi%20University%20of%20Science%20and%20Technology/FPT_AIC/code does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [57]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating base model...')
base_name = 'base40M-textvec'
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

# print('creating upsample model...')
# upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
# upsampler_model.eval()
# upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

# print('downloading upsampler checkpoint...')
# upsampler_model.load_state_dict(load_checkpoint('upsample', device))

creating base model...
creating upsample model...
downloading base checkpoint...
downloading upsampler checkpoint...


<All keys matched successfully>

In [58]:
sampler = PointCloudSampler(
    device=device,
    models=[base_model, upsampler_model],
    diffusions=[base_diffusion, upsampler_diffusion],
    num_points=[1024, 4096 - 1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0, 0.0],
    model_kwargs_key_filter=('texts', ''), # Do not condition the upsampler at all
)

In [77]:
sampler = PointCloudSampler(
  device=device,
  models=[base_model],
  diffusions=[base_diffusion],
  num_points=[1024],
  aux_channels=['R', 'G', 'B'],
  guidance_scale=[3.0],
  model_kwargs_key_filter=['texts'],
  use_karras = [True],
  karras_steps = [64],
  sigma_min = [1e-3],
  sigma_max = [120],
  s_churn = [3],
)

In [11]:
print(samples)

tensor([[[-1.4755e-02,  8.3027e-03,  2.0180e-03,  ...,  8.1520e-03,
          -6.8459e-02, -2.8924e-02],
         [ 2.6321e-02,  3.2393e-02,  8.3271e-02,  ...,  4.9217e-02,
          -4.6284e-02,  8.7147e-03],
         [-2.3000e-01, -2.5585e-01, -1.5788e-01,  ..., -3.8034e-01,
          -1.5378e-01, -1.7454e-01],
         [ 1.6914e+02,  1.5722e+02,  1.4277e+02,  ...,  1.3945e+02,
           1.6102e+02,  1.6777e+02],
         [ 3.9010e+01,  9.5830e+00,  3.6495e+01,  ...,  4.5379e+00,
           2.3765e+01,  2.8359e+01],
         [-2.0927e+00,  1.1388e+01, -5.7164e+00,  ..., -2.4936e+01,
          -3.7699e+00, -1.7856e+01]]])


In [30]:
# Set a prompt to condition on.
prompt = 'a red motorcycle'

# Produce a sample from the model.
samples = None
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(texts=[prompt]))):
    samples = x

0it [00:00, ?it/s]

None


KeyboardInterrupt: 

In [63]:
%cd Point_E/

c:\Users\X1G6\OneDrive - Hanoi University of Science and Technology\FPT_AIC\code\Point_E


In [80]:
pc = PointCloud.load('point_e\examples\example_data\pc_corgi.npz')
pc = pc.add_noise(0.05)
pc = pc.random_sample(1024)
noise_after_drag = pc.to_tensor()
noise_after_drag
# plot_3D(pc)


tensor([[[ 7.3422e-02, -2.4696e-03, -1.5381e-01,  ...,  9.1670e-02,
           1.5906e-01, -2.6842e-02],
         [-1.7574e-01,  6.6889e-02, -2.7845e-01,  ..., -1.9984e-01,
          -4.1446e-02,  2.4355e-01],
         [ 3.7412e-02, -2.2694e-01,  1.2901e-01,  ..., -2.3200e-01,
          -1.3540e-01, -5.1726e-02],
         [ 2.4500e+02,  2.4500e+02,  2.3700e+02,  ...,  1.7200e+02,
           2.4500e+02,  2.4500e+02],
         [ 1.8400e+02,  1.8300e+02,  2.3700e+02,  ...,  1.7200e+02,
           1.8300e+02,  1.8300e+02],
         [ 1.1900e+02,  1.1900e+02,  2.3700e+02,  ...,  1.7200e+02,
           1.1900e+02,  1.1900e+02]]])

In [74]:
x = torch.rand(1, 1024, 3, device=device)
x.dtype

torch.float32

In [79]:
# Set a prompt to condition on.
prompt = 'a dog' # @param {type:"string"}

# Produce a sample from the model.
samples = None
iterSample = []
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(texts=[prompt]),noise_after_drag  = noise_after_drag)):
  samples = x
  iterSample.append(x)


0it [00:00, ?it/s]

None
tensor([[[ 5.0000e-01,  5.0000e-01,  5.0000e-01,  ...,  5.0000e-01,
           5.0000e-01,  5.0000e-01],
         [ 5.0000e-01,  5.0000e-01,  5.0000e-01,  ...,  5.0000e-01,
           5.0000e-01,  5.0000e-01],
         [-4.0513e-01, -5.9026e-02,  4.4734e-01,  ...,  5.0000e-01,
           9.1195e-02,  2.7137e-01],
         [ 2.5500e+02,  2.5500e+02,  2.5500e+02,  ...,  2.5500e+02,
           2.5500e+02,  2.5500e+02],
         [ 2.5500e+02,  2.5500e+02,  2.5500e+02,  ...,  2.5500e+02,
           2.5500e+02,  2.5500e+02],
         [ 2.5500e+02,  2.5500e+02,  2.5500e+02,  ...,  2.5500e+02,
           2.5500e+02,  2.5500e+02]]])
tensor([[[-3.9737e-01, -1.1434e-02,  1.4273e-01,  ...,  5.0000e-01,
           2.9883e-01,  2.5710e-01],
         [ 5.0000e-01,  5.0000e-01,  5.0000e-01,  ...,  5.0000e-01,
           5.0000e-01,  5.0000e-01],
         [ 3.6800e-01,  4.3755e-02,  5.0000e-01,  ...,  5.0000e-01,
           5.0000e-01,  5.0000e-01],
         [ 2.5500e+02,  2.5500e+02,  2.5500e+02,

KeyboardInterrupt: 

In [47]:
torch.randn(1, 6, 1024).shape

torch.Size([1, 6, 1024])

In [49]:
# Produce a sample from the model.
prompt 
samples = None
iterSample1 = []
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(texts=[prompt]), noise_after_drag = torch.randn(1, 6, 1024))):
  samples = x
  iterSample1.append(x)


0it [00:00, ?it/s]

None
tensor([[[  0.5000,   0.5000,   0.5000,  ...,  -0.5000,   0.5000,   0.5000],
         [ -0.5000,   0.5000,   0.5000,  ...,   0.5000,   0.5000,  -0.5000],
         [  0.5000,   0.5000,   0.3880,  ...,  -0.5000,  -0.5000,   0.5000],
         [286.2378, 255.0000, 255.0000,  ..., 255.0000,   0.0000, 196.8022],
         [  0.0000,   0.0000, 180.5527,  ...,  45.0758, 255.0000,   0.0000],
         [255.0000, 255.0000, 160.2758,  ..., 255.0000, 255.0000, 255.0000]]])
tensor([[[  0.5000,   0.5000,  -0.5000,  ...,  -0.5000,   0.5000,   0.3987],
         [ -0.5000,   0.5000,  -0.5000,  ...,   0.5000,   0.5000,  -0.5000],
         [ -0.2877,  -0.5000,   0.6799,  ...,  -0.5000,  -0.5000,  -0.5000],
         [284.0267, 193.5768, 255.0000,  ..., 255.0000,   0.0000,   0.0000],
         [255.0000,  39.8900, 255.0000,  ..., 175.8471, 255.0000,   0.0000],
         [  0.0000, 255.0000, 255.0000,  ..., 255.0000, 255.0000,  26.5336]]])
tensor([[[ 5.0000e-01,  5.0000e-01,  5.0000e-01,  ...,  1.9793e-01,

KeyboardInterrupt: 

In [50]:
pc = sampler.output_to_point_clouds(iterSample1[-1])[0]
plot_3D(pc)

In [8]:
sampler

In [ ]:
pc = sampler.output_to_point_clouds(samples)[0]
fig = plot_point_cloud(pc, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))